<a href="https://colab.research.google.com/github/KhoaTran104a10/Ung-dung-phan-tan-Nhom4/blob/main/BTL_UDPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Chạy ô này đầu tiên
!pip install tinydb pandas -q


In [ ]:
# Node + Aggregator + helper (Colab-ready)
from tinydb import TinyDB, where
from uuid import uuid4
from datetime import datetime, timezone, timedelta
import random, time, os, zipfile, pandas as pd

# Optional helper for nicer display in some environments
try:
    from caas_jupyter_tools import display_dataframe_to_user
except Exception:
    display_dataframe_to_user = None

class Node:
    """Represents a Smart Bin node with a local TinyDB storage."""
    def __init__(self, bin_id, db_path=None):
        self.bin_id = bin_id
        self.db_path = db_path or f'{bin_id}_local.json'
        self.db = TinyDB(self.db_path)

    def sense(self, weight_kg=None, fill_level=None, waste_type=None, timestamp=None):
        rec = {
            'id': str(uuid4()),
            'bin_id': self.bin_id,
            'timestamp': (timestamp or datetime.now(timezone(timedelta(hours=7))).isoformat()),
            'weight_kg': weight_kg if weight_kg is not None else round(random.uniform(0.1, 5.0), 3),
            'fill_level': fill_level if fill_level is not None else random.randint(5, 100),
            'type': waste_type or random.choice(['general','recyclable','organic']),
            'synced': False
        }
        self.db.insert(rec)
        return rec

    def get_unsynced(self):
        return [r for r in self.db.all() if not r.get('synced')]

    def mark_synced(self, ids):
        for _id in ids:
            self.db.update({'synced': True}, where('id') == _id)

    def push(self, aggregator):
        unsynced = self.get_unsynced()
        if not unsynced:
            return 0, {'ok': True, 'received': 0}
        try:
            resp = aggregator.ingest(unsynced)
        except Exception as e:
            return 0, {'ok': False, 'error': str(e)}
        if resp.get('ok'):
            ids = [d['id'] for d in unsynced]
            self.mark_synced(ids)
            return len(unsynced), resp
        return 0, resp

class Aggregator:
    """Aggregator collects records from nodes and stores in a central TinyDB."""
    def __init__(self, db_path='aggregator_db.json', simulate_failure=False):
        self.db_path = db_path
        self.db = TinyDB(self.db_path)
        self.simulate_failure = simulate_failure

    def ingest(self, docs):
        if self.simulate_failure and random.random() < 0.25:
            raise RuntimeError('Simulated network/aggregator failure')
        received = 0
        for d in docs:
            existing = self.db.search(where('id') == d['id'])
            if not existing:
                doc = {k: v for k, v in d.items() if k != 'synced'}
                self.db.insert(doc)
                received += 1
        return {'ok': True, 'received': received}

    def all(self):
        return self.db.all()

    def to_dataframe(self):
        rows = self.all()
        if not rows:
            return pd.DataFrame()
        df = pd.DataFrame(rows)
        try:
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        except Exception:
            pass
        return df.sort_values('timestamp').reset_index(drop=True)


In [ ]:
def run_simulation(num_nodes=3, readings_per_node=5, push_interval=0.5, simulate_failure=False):
    # Clean prior files to keep workspace tidy
    for f in os.listdir('.'):
        if f.endswith('_local.json') or f == 'aggregator_db.json' or f == 'smartbin_dbs.zip':
            try:
                os.remove(f)
            except:
                pass

    agg = Aggregator(simulate_failure=simulate_failure)
    nodes = [Node(f'bin-{i+1}', db_path=f'bin-{i+1}_local.json') for i in range(num_nodes)]
    temp_files = [n.db_path for n in nodes] + [agg.db_path]

    for r in range(readings_per_node):
        # Each node senses one reading
        for node in nodes:
            node.sense()
        # Then each node attempts to push to aggregator
        for node in nodes:
            pushed, resp = node.push(agg)
            print(f'Node {node.bin_id} pushed {pushed} records -> agg received {resp.get("received")}, ok={resp.get("ok")}')
        time.sleep(push_interval)

    df = agg.to_dataframe()
    if display_dataframe_to_user and not df.empty:
        display_dataframe_to_user('Aggregator records', df)
    else:
        print('\n--- Aggregator stored records (sample) ---')
        print(df.head(200).to_string(index=False))

    # Save DBs to zip for download
    zip_name = 'smartbin_dbs.zip'
    with zipfile.ZipFile(zip_name, 'w') as zf:
        for f in temp_files:
            if os.path.exists(f):
                zf.write(f)
    print(f'\nDatabase files saved to: {zip_name}')
    return nodes, agg

# Example run (tweak parameters as you like)
nodes, agg = run_simulation(num_nodes=4, readings_per_node=6, push_interval=0.2, simulate_failure=False)

# Show aggregator dataframe
final_df = agg.to_dataframe()
final_df.head(50)


Node bin-1 pushed 1 records -> agg received 1, ok=True
Node bin-2 pushed 1 records -> agg received 1, ok=True
Node bin-3 pushed 1 records -> agg received 1, ok=True
Node bin-4 pushed 1 records -> agg received 1, ok=True
Node bin-1 pushed 1 records -> agg received 1, ok=True
Node bin-2 pushed 1 records -> agg received 1, ok=True
Node bin-3 pushed 1 records -> agg received 1, ok=True
Node bin-4 pushed 1 records -> agg received 1, ok=True
Node bin-1 pushed 1 records -> agg received 1, ok=True
Node bin-2 pushed 1 records -> agg received 1, ok=True
Node bin-3 pushed 1 records -> agg received 1, ok=True
Node bin-4 pushed 1 records -> agg received 1, ok=True
Node bin-1 pushed 1 records -> agg received 1, ok=True
Node bin-2 pushed 1 records -> agg received 1, ok=True
Node bin-3 pushed 1 records -> agg received 1, ok=True
Node bin-4 pushed 1 records -> agg received 1, ok=True
Node bin-1 pushed 1 records -> agg received 1, ok=True
Node bin-2 pushed 1 records -> agg received 1, ok=True
Node bin-3

,id,bin_id,timestamp,weight_kg,fill_level,type
0,3f5ea098-c8d7-41f9-88ae-16ec0c3d5960,bin-1,2025-10-08 09:55:15.111185+07:00,2.090,19,organic
1,835d8471-ba41-40fd-8401-ac96b724b8f9,bin-2,2025-10-08 09:55:15.115128+07:00,1.740,36,organic
2,ec800424-6508-433f-a41b-a0d46b103c99,bin-3,2025-10-08 09:55:15.119394+07:00,2.821,83,general
3,e9d7b856-2b33-4491-9600-7007d85ee05a,bin-4,2025-10-08 09:55:15.123068+07:00,2.398,77,general
4,aedc6641-693f-44df-b31f-8b6445045b82,bin-1,2025-10-08 09:55:15.355416+07:00,4.539,13,recyclable
5,65c85d1f-463e-41f1-be7a-376946c0a1f9,bin-2,2025-10-08 09:55:15.359672+07:00,4.688,31,organic
6,080cde59-a121-4847-a502-d58fd74f9285,bin-3,2025-10-08 09:55:15.363474+07:00,1.589,72,general
7,088d0225-78f4-48d9-9c90-a5bf949d8f88,bin-4,2025-10-08 09:55:15.366830+07:00,1.162,63,organic
8,57b993f5-a994-45c2-afd3-edf0b331e89c,bin-1,2025-10-08 09:55:15.598701+07:00,2.722,45,recyclable
9,6fc735f1-06c5-49a6-8dbf-08288be7c95b,bin-2,2025-10-08 09:55:15.603035+07:00,3.022,18,organic
